In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import yfinance as yf
import copy
import random
from pymongo import MongoClient
import sys
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

load_dotenv()
mongo_client = MongoClient(os.environ.get("MONGO_URI"))
stock_db = mongo_client["stock_db"]

#### Chuẩn bị các dữ liệu

##### Các dữ liệu dùng để làm map tham chiếu

In [7]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'order'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'full_name'],axis=1)
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='name_map').drop(columns=['order', 'full_name'],axis=1)
group_map_dict = group_map.set_index('code')['group'].to_dict()


#Tạo các danh sách nhóm trong mỗi cách chia cổ phiếu
all_stock_key_list = [key for key, value in group_map_dict.items() if value == 'all']
industry_name_list = [key for key, value in group_map_dict.items() if value in ['hsA', 'hsB', 'hsC', 'hsD']]
industry_perform_list = [key for key, value in group_map_dict.items() if value == 'hs']
marketcap_group_list = [key for key, value in group_map_dict.items() if value == 'cap']

#Tạo danh danh key cho tổng tất cả các nhóm
group_stock_key_list = all_stock_key_list + industry_name_list + industry_perform_list + marketcap_group_list

In [8]:
def get_file_name_list(folder_path):
    file_name_list = []
    files = os.listdir(folder_path)
    for file in files:
        file_name_list.append(file[:-4])
    return file_name_list

def filter_market_file_name_list(file_name_list):
    filtered_list = [item for item in file_name_list if not (item.endswith('_AC') or item.endswith('_CC'))]
    return filtered_list

eod_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\stock"
eod_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\index"
eod_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\futures"
itd_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\stock"
itd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\index"
itd_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\futures"
nn_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\foreign"
td_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\prop"
nntd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\market"
other_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\other"

In [9]:
#Tạo dict map thời gian và số lượng cổ phiếu
quarter_map = pd.read_excel("../.xlsx/quarter_stock_map.xlsx", sheet_name='quarter_map')
quarter_map_dict = quarter_map.set_index('index').apply(lambda row: row.tolist(), axis=1).to_dict()
quarter_name_list = list(quarter_map_dict.keys())

#Xoá đi quý hiện tại để chỉ tính toán tới quý trước đó
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
        previous_year = year - 1
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q1"
        previous_year = year
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
        previous_year = year
    else:
        quarter = "q4"
        previous_quarter = "q3"
        previous_year = year
    
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        return f'{previous_quarter}_{previous_year}'

#Lấy ra list cổ phiếu của giai đoạn hiện tại
quarter_stock_list = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='quarter_stock_list')

current_quarter_stock_list = list(set(get_file_name_list(itd_stock_folder_path)) 
                                  & set(quarter_stock_list[get_quarter('current_quarter')].dropna().tolist()))

total_stock_list = list(set(get_file_name_list(itd_stock_folder_path)) & set(quarter_stock_list['all'].dropna().tolist()))

#Lấy ra khoảng thời gian tính toán cho quý này và quý trước
calculate_time_span = [quarter_map_dict['q2_2020'][0], quarter_map_dict[get_quarter('current_quarter')][1]]
current_quarter_span = [quarter_map_dict[get_quarter('current_quarter')][0], quarter_map_dict[get_quarter('current_quarter')][1]]
previous_quarter_span = [quarter_map_dict[get_quarter('previous_quarter')][0], quarter_map_dict[get_quarter('previous_quarter')][1]]

##### Các biến thời gian

In [10]:
def decode_data(file_path):
    # Đọc dữ liệu vào một numpy array
    data = np.fromfile(file_path, dtype=np.uint8)

    # Giả định kích thước mỗi bản ghi (có thể thay đổi tùy theo cấu trúc tệp thực tế)
    record_size = 32  # Giả định
    num_records = len(data) // record_size

    # Số lượng cột dữ liệu (bao gồm ngày, thời gian và các giá trị int32 còn lại)
    num_columns = record_size // 4  # Mỗi giá trị int32 chiếm 4 byte

    # Sử dụng numpy để cắt và giải mã dữ liệu hiệu quả hơn
    # Tạo một numpy array để chứa các giá trị int32 và float32
    raw_data = data.reshape(num_records, record_size // 4, 4)

    # Giải mã ngày và thời gian (int32) ở cột 0 và 1, các cột còn lại là float32
    int_data = raw_data[:, :2].view(np.int32)  # Giải mã int32 (2 cột)
    float_data = raw_data[:, 2:].view(np.float32)  # Giải mã float32 (các cột còn lại)

    # Kết hợp dữ liệu
    records = np.hstack((int_data, float_data))

    # Đảm bảo rằng dữ liệu là 2D
    records = records.reshape(num_records, num_columns)

    # Đảo ngược lại dữ liệu trước khi chuyển thành DataFrame
    records = records[::-1]

    # Chuyển đổi thành DataFrame và loại bỏ dòng đầu tiên
    df = pd.DataFrame(records, columns=[f"Col_{i}" for i in range(num_columns)])
    return df  # Loại bỏ đi dòng dữ liệu đầu tiên không cần thiết

In [11]:
def clean_eod_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1', 'Col_7'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    
    return df_clean.reset_index(drop=True)

def clean_itd_data(df_raw):
    #Lọc ra đúng 1 ngày dữ liệu cuối cùng
    df_raw = df_raw[df_raw['Col_0'] == max(df_raw['Col_0'])]
    #Tạo cột date-time mới từ 2 cột date và time cũ
    df_raw['date'] = df_raw['Col_0'].astype(int).astype(str) + ' ' + df_raw['Col_1'].astype(int).astype(str)
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_0', 'Col_1', 'Col_7'])
    #Sắp xếp lại thứ tự các cột
    df_clean = df_clean[['date'] + [f"Col_{i}" for i in range(2, len(df_clean.columns)+1)]]
    # #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['date'] = pd.to_datetime(df_clean['date'], format='%Y%m%d %H%M%S')
    # #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    #Làm tròn khung thời gian tới 5 phút
    df_clean['date'] = df_clean['date'].dt.floor('1T')
    df_clean = df_clean.set_index("date").resample("1T", closed='right', label='right').agg({    
        "open": "first",  
        "high": "max",  
        "low": "min", 
        "close": "last",  
        "volume": "sum"   
    }).dropna().reset_index()

    return df_clean.sort_values(by="date", ascending=False).reset_index(drop=True)

In [12]:
#Khởi tạo vnindex_series để xác định ngày hiện tại
vnindex_series = clean_eod_data(decode_data(eod_index_folder_path + '\\VNINDEX.dat'))['date']

#Tạo date_series cho thời gian tính toán
date_series = pd.DataFrame(vnindex_series).rename(columns={0:'date'})
date_series = date_series[(date_series['date'] >= calculate_time_span[0]) & (date_series['date'] <= calculate_time_span[1])]

#Xác định ngày hiện tại
today = vnindex_series.iloc[0]

#Xác định thời gian hiện tại
current_time = clean_itd_data(decode_data(itd_index_folder_path + '\\VNXALL.dat'))['date'].iloc[0]

#Khởi tạo time_series bao gồm tất cả khung thời gian của ngày hiện tại
time_series_list = []
time_series_list.extend(pd.date_range(start=f'{today} 09:00:00', end=f'{today} 11:25:00', freq='1T'))
time_series_list.extend(pd.date_range(start=f'{today} 13:00:00', end=f'{today} 14:59:00', freq='1T'))
time_series = pd.DataFrame(time_series_list).rename(columns={0:'date'})

#Điều chỉnh lại time_series bỏ đi các hàng thời gian chưa có dữ liệu
time_series = time_series.loc[time_series['date'].dt.time <= current_time.time()].sort_values('date', ascending=False).reset_index(drop=True)

#Khởi tạo khung thời gian bắt đầu từ 9h00 để vẽ các biểu đồ
itd_series = pd.DataFrame(time_series_list).rename(columns={0:'date'}).sort_values('date', ascending=False).reset_index(drop=True)

#Đọc file phân bổ thanh khoản trong phiên
itd_time_percent = pd.read_excel('../.xlsx/itd_time_percent.xlsx')
#Chuyển đổi ngày thành này hôm nay
itd_time_percent['date'] = itd_time_percent['date'].apply(lambda x: today.replace(hour=x.hour, minute=x.minute, second=x.second))
#Khởi tạo hệ số thời gian
current_time_percent = itd_time_percent[itd_time_percent['date'] == current_time]['percent'].item()

##### Đọc dữ liệu nhóm từ MongoDB

In [13]:
def get_mongo_df(df_name, find_query=None, projection=None):
    # Truy cập collection
    collection = stock_db[df_name]
    # Nếu không truyền vào find_query thì mặc định lấy tất cả document
    if find_query is None:
        find_query = {}
    # Nếu không truyền vào projection thì mặc định loại bỏ trường _id
    if projection is None:
        projection = {"_id": 0}
    # Thực hiện lệnh find với điều kiện và projection đã cho
    docs = collection.find(find_query, projection)
    # Chuyển đổi kết quả sang DataFrame và trả về
    df = pd.DataFrame(list(docs))
    return df

In [14]:
#Đọc dữ liệu lịch sử cổ phiếu
eod_stock_df = get_mongo_df("eod_stock", projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, "t0_score": 1, "t5_score": 1, "WFIBO_0618": 1, "MFIBO_0618": 1, "WFIBO_0382": 1})
history_stock_df = get_mongo_df("history_stock", projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, "t0_score": 1, "t5_score": 1, "WFIBO_0618": 1, "MFIBO_0618": 1, "WFIBO_0382": 1})
full_stock_df = pd.concat([eod_stock_df, history_stock_df], axis=0).reset_index(drop=True)

#Đọc dữ liệu lịch sử cổ phiếu
eod_index_df = get_mongo_df("eod_index", projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, "t0_score": 1, "t5_score": 1, "WFIBO_0618": 1, "MFIBO_0618": 1, "WFIBO_0382": 1})
history_index_df = get_mongo_df("history_index", find_query={"type": "spot"}, projection={"_id": 0, "ticker": 1, "date": 1, "open": 1, "high": 1, "low": 1, "close": 1, 'volume': 1, "t0_score": 1, "t5_score": 1, "WFIBO_0618": 1, "MFIBO_0618": 1, "WFIBO_0382": 1})
full_index_df = pd.concat([eod_index_df, history_index_df], axis=0).reset_index(drop=True)

#Đọc dữ liệu điểm dòng tiền nhóm cổ phiếu
eod_group_df = get_mongo_df("eod_group")
history_group_df = get_mongo_df("history_group")
full_group_df = pd.concat([eod_group_df, history_group_df], axis=0).reset_index(drop=True)

#Đọc dữ liệu điểm dòng tiền âm dương nhóm cổ phiếu
eod_signed_group_df = get_mongo_df("eod_signed_group")
history_signed_group_df = get_mongo_df("history_signed_group")
itd_signed_group_df = get_mongo_df("itd_signed_group")
full_signed_group_df = pd.concat([eod_signed_group_df, history_signed_group_df], axis=0).reset_index(drop=True)

#Đọc dữ liệu ms lịch sử
eod_ms_chart_df = get_mongo_df("eod_ms_chart")
history_ms_chart_df = get_mongo_df("history_ms_chart")
full_ms_chart_df = pd.concat([eod_ms_chart_df, history_ms_chart_df], axis=0).reset_index(drop=True)

In [15]:
#Bảng phân loại cổ phiếu
stock_classification_df = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='classification')

def round_price_by_step(df, value_column, step_size_column):
    # Tạo một bản sao để tránh thay đổi df gốc
    result = df[value_column].copy()
    
    # Trường hợp step_size > 0: làm tròn theo bội số
    mask = df[step_size_column] > 0
    if mask.any():
        result.loc[mask] = (df.loc[mask, value_column] / df.loc[mask, step_size_column]).round() * df.loc[mask, step_size_column]
    
    return result

#Tạo lại stock_dict từ dữ liệu monggoDB
stock_dict = {}
for ticker in full_stock_df['ticker'].unique():
    temp_df = full_stock_df[full_stock_df['ticker'] == ticker].reset_index(drop=True)
    temp_df['t0_score'] = temp_df['t0_score'].fillna(0)
    temp_df['t5_score'] = temp_df['t5_score'].fillna(0)
    temp_df['exchange'] = stock_classification_df.set_index('ticker').at[ticker, 'exchange']
    temp_df['price_step'] = temp_df.apply(lambda x: 0.01 if (x['exchange'] == 'HSX') & (x['close'] < 10) else (0.05 if (x['exchange'] == 'HSX') & (x['close'] < 50) else 0.1), axis=1)
    temp_df['pct_change'] = temp_df['close'][::-1].pct_change()[::-1].ffill()

    #Các cột biến động giá cần dùng
    temp_df['lowest_1'] = temp_df['low'][::-1].rolling(window=1, min_periods=1).min()[::-1]
    temp_df['diff_lowest1'] = (temp_df['close'] - temp_df['lowest_1'])/temp_df['lowest_1']
    temp_df['lowest_10'] = temp_df['low'][::-1].rolling(window=10, min_periods=1).min()[::-1]
    temp_df['diff_lowest10'] = (temp_df['close'] - temp_df['lowest_10'])/temp_df['lowest_10']
    temp_df['highest_10'] = temp_df['high'][::-1].rolling(window=10, min_periods=1).max()[::-1]
    temp_df['diff_highest10'] = (temp_df['close'] - temp_df['highest_10'])/temp_df['highest_10']
    
    #Các cột khối lượng giao dịch để lọc
    temp_df['min_vol'] = temp_df['volume'][::-1].rolling(window=10, min_periods=1).min()[::-1]
    temp_df['mean_vol'] = temp_df['volume'][::-1].rolling(window=60, min_periods=1).mean()[::-1]

    #Các cột giá trị giao dịch để lọc
    temp_df['value_traded'] = temp_df['close']*temp_df['volume']
    temp_df['min_value'] = temp_df['value_traded'][::-1].rolling(window=10, min_periods=1).min()[::-1]
    temp_df['mean_value'] = temp_df['value_traded'][::-1].rolling(window=60, min_periods=1).mean()[::-1]

    #Các cột tính các mốc giá mua bán khuyến nghị trong phiên
    temp_df['calculate_high'] = np.maximum(temp_df['high'], temp_df['close'].shift(-1).ffill()*1.015)
    temp_df['calculate_low'] = np.minimum(temp_df['low'], temp_df['close'].shift(-1).ffill()*0.985)

    temp_df['trade_high'] = temp_df['calculate_high'] - (temp_df['calculate_high'] - np.minimum(temp_df['open'], temp_df['close']))*0.382
    temp_df['trade_mean'] = (temp_df['close'] + temp_df['open'])/2
    temp_df['trade_low'] = temp_df['calculate_low'] + (np.maximum(temp_df['open'], temp_df['close']) - temp_df['calculate_low'])*0.382

    temp_df['trade_high'] = round_price_by_step(temp_df, 'trade_high', 'price_step')
    temp_df['trade_mean'] = round_price_by_step(temp_df, 'trade_mean', 'price_step')
    temp_df['trade_low'] = round_price_by_step(temp_df, 'trade_low', 'price_step')

    # Các giá trị cơ sở khi không có biến động giá
    base_high = 0.30
    base_mean = 0.40
    base_low = 0.30
    
    # Hệ số điều chỉnh với tỉ lệ không đối xứng
    high_factor = 4.0  # Hệ số tăng mạnh cho portion_high khi giá tăng
    low_factor = 4.0   # Hệ số tăng mạnh cho portion_low khi giá giảm
    mean_factor = 2.0  # Hệ số điều chỉnh nhẹ cho portion_mean
    
    # Khởi tạo các cột portion mặc định
    temp_df['portion_high'] = base_high
    temp_df['portion_mean'] = base_mean
    temp_df['portion_low'] = base_low
    
    # Xử lý khi giá tăng (pct_change > 0)
    increase_mask = temp_df['pct_change'] > 0
    if increase_mask.any():
        # portion_high tăng nhiều
        temp_df.loc[increase_mask, 'portion_high'] = base_high + temp_df.loc[increase_mask, 'pct_change'] * high_factor
        # portion_mean giảm nhẹ
        temp_df.loc[increase_mask, 'portion_mean'] = base_mean - temp_df.loc[increase_mask, 'pct_change'] * mean_factor * 0.5
        # portion_low giảm ít
        temp_df.loc[increase_mask, 'portion_low'] = 1 - temp_df.loc[increase_mask, 'portion_high'] - temp_df.loc[increase_mask, 'portion_mean']

    # Xử lý khi giá giảm (pct_change < 0)
    decrease_mask = temp_df['pct_change'] < 0
    if decrease_mask.any():
        # portion_low tăng nhiều 
        temp_df.loc[decrease_mask, 'portion_low'] = base_low - temp_df.loc[decrease_mask, 'pct_change'] * low_factor
        # portion_mean giảm nhẹ
        temp_df.loc[decrease_mask, 'portion_mean'] = base_mean + temp_df.loc[decrease_mask, 'pct_change'] * mean_factor * 0.5
        # portion_high giảm ít
        temp_df.loc[decrease_mask, 'portion_high'] = 1 - temp_df.loc[decrease_mask, 'portion_low'] - temp_df.loc[decrease_mask, 'portion_mean']
    
    # Đảm bảo tổng tỉ trọng = 1
    temp_df['portion_mean'] = 1 - temp_df['portion_high'] - temp_df['portion_low']

    #Tính giá cuối cùng để tính hiệu suất mua bán
    temp_df['trade_price'] = temp_df['portion_high']*temp_df['trade_high'] + temp_df['portion_mean']*temp_df['trade_mean'] + temp_df['portion_low']*temp_df['trade_low']
    temp_df['trade_price'] = round_price_by_step(temp_df, 'trade_price', 'price_step')
    
    stock_dict[ticker] = temp_df

#Dict tất cả các cổ phiếu trong tất cả giai đoạn
quarter_stock_df = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='quarter_stock_list')
quarter_stock_dict = {}
for quarter in quarter_name_list:
    quarter_stock_dict[quarter] = {k: v[(v['date'] >= quarter_map_dict[quarter][0]) & 
                                      (v['date'] <= quarter_map_dict[quarter][1])].reset_index(drop=True)
                                      for k, v in copy.deepcopy(stock_dict).items()
                                      if k in quarter_stock_df[quarter].dropna().tolist()}
    
#Dict chứa date series cho từng giai đoạn
quarter_date_series_dict = {}
for quarter in quarter_name_list:
    quarter_date_series_dict[quarter] = date_series[(date_series['date'] >= quarter_map_dict[quarter][0]) & 
                                                       (date_series['date'] <= quarter_map_dict[quarter][1])].reset_index(drop=True)
    
#Chuyển đổi thời gian trong quarter_map_dict sang pd.Timestamp để có thể so sánh với ngày trong df
quarter_timestamp_map_dict = {}
for key, value in quarter_map_dict.items():
    quarter_timestamp_map_dict[key] = (pd.Timestamp(value[0]), pd.Timestamp(value[1]), value[2])

#### Tín hiệu toàn thị trường

##### Tạo bảng market phase

- Các hàm tính toán

In [53]:
def calculate_market_ms(ms_df):
    # Tạo bản sao của DataFrame để không làm thay đổi dữ liệu gốc
    df = ms_df.copy()

    # --- Tính toán các cột shift cho trend_5p ---
    df['5p_shift1'] = df['trend_5p'].shift(-1)
    df['5p_shift2'] = df['trend_5p'].shift(-2)
    df['5p_shift5'] = df['trend_5p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1)
    df['5p_diff1'] = df['trend_5p'][::-1].diff()[::-1]
    df['5p_diff5'] = df['trend_5p'] - df['5p_shift5']
    
    # --- Tính toán các cột shift cho trend_20p ---
    df['20p_shift1'] = df['trend_20p'].shift(-1)
    df['20p_shift2'] = df['trend_20p'].shift(-2)
    df['20p_shift5'] = df['trend_20p'][::-1].rolling(window=5, min_periods=1).mean()[::-1].shift(-1)
    df['20p_diff1'] = df['trend_20p'][::-1].diff()[::-1]
    df['20p_diff5'] = df['trend_20p'] - df['20p_shift5']

    # --- Tính toán các cột shift cho trend_60p ---
    df['60p_shift1'] = df['trend_60p'].shift(-1)
    df['60p_shift2'] = df['trend_60p'].shift(-2)

    # --- Tín hiệu mua (buy) ---
    # So sánh giá trị hiện tại với các giá trị dịch chuyển
    df['buy_ms5p_check'] = ((df['trend_5p'] > df['5p_shift1']) & (df['trend_5p'] > df['5p_shift5'])).astype(int)
    df['buy_ms20p_check'] = ((df['trend_20p'] > df['20p_shift1']) & (df['trend_20p'] > df['20p_shift5'])).astype(int)
    df['buy_ms60p_check'] = ((df['trend_60p'] > df['60p_shift1']) & (df['trend_60p'] > df['60p_shift2'])).astype(int)

    # Kiểm tra xem giá có ở mức quá cao không
    df['buy_overprice_check'] = ((df['5p_diff1'] > 0.4) | ((df['trend_5p'] < 0.9) & (df['trend_20p'] < 0.9))).astype(int)

    # Kiểm tra giá trị biến động của xu hướng
    buy_threshold_5p = 0.3
    buy_threshold_20p = 0.2
    df['buy_msvalue_check'] = ( (df['5p_diff1'] > 0.4) | 
            (((df['5p_shift1'] > buy_threshold_5p) & (df['trend_5p'] > buy_threshold_5p)) | ((df['5p_shift1'] < buy_threshold_5p) & (df['trend_5p'] > buy_threshold_5p) & (df['5p_diff1']*0.3 < (df['trend_5p'] - buy_threshold_5p)))) &
            (((df['20p_shift1'] > buy_threshold_20p) & (df['trend_20p'] > buy_threshold_20p)) | ((df['20p_shift1'] < buy_threshold_20p) & (df['trend_20p'] > buy_threshold_20p) & (df['20p_diff1']*0.3 < (df['trend_20p'] - buy_threshold_20p))))
        ).astype(int)

    # Tính tín hiệu mua dựa trên tất cả các điều kiện
    df['buy_check'] = df.apply(
        lambda x: 1 if (x['buy_ms5p_check'] == 1 and x['buy_ms20p_check'] == 1 and 
                        x['buy_ms60p_check'] == 1 and x['buy_overprice_check'] == 1 and 
                        x['buy_msvalue_check'] == 1)
                  else (2 if x['buy_overprice_check'] == 0 else 0),
        axis=1
    )

    # --- Tín hiệu bán (sell) ---
    df['sell_ms5p_check'] = ((df['trend_5p'] < df['5p_shift1']) | (df['trend_5p'] < df['5p_shift2'])).astype(int)
    df['sell_ms20p_check'] = ((df['trend_20p'] < df['20p_shift1']) | (df['trend_20p'] < df['20p_shift2'])).astype(int)

    # Kiểm tra xem giá có ở mức quá thấp không (underprice)
    df['sell_underprice_check'] = ((df['trend_5p'] > 0.22) | (df['trend_20p'] > 0.25)).astype(int)

    # Kiểm tra giá trị biến động cho tín hiệu bán
    sell_threshold_5p = 0.4
    sell_threshold_20p = 0.5
    df['sell_msvalue_check'] = ( (df['5p_diff1'] < -0.4) |
                                
        (((df['5p_shift1'] < sell_threshold_5p) & (df['trend_5p'] < sell_threshold_5p)) | ((df['5p_shift1'] > sell_threshold_5p) & (df['trend_5p'] < sell_threshold_5p) & (-df['5p_diff1']*0.3 < (sell_threshold_5p - df['trend_5p'])))) &
        (((df['20p_shift1'] < sell_threshold_20p) & (df['trend_20p'] < sell_threshold_20p)) | ((df['20p_shift1'] > sell_threshold_20p) & (df['trend_20p'] < sell_threshold_20p) & (-df['20p_diff1']*0.3 < (sell_threshold_20p - df['trend_20p'])))) &
        
        (df['trend_60p'] < 0.7)
    ).astype(int)
    
    # Tính tín hiệu bán dựa trên tất cả các điều kiện
    df['sell_check'] = df.apply(
        lambda x: 1 if (x['sell_ms5p_check'] == 1 and x['sell_ms20p_check'] == 1 and 
                        x['sell_underprice_check'] == 1 and x['sell_msvalue_check'] == 1)
                  else (2 if x['sell_underprice_check'] == 0 else 0),
        axis=1
    )

    # return df[['date','trend_5p','trend_20p','trend_60p','buy_ms5p_check','buy_ms20p_check','buy_ms60p_check','buy_overprice_check','buy_msvalue_check','sell_ms5p_check','sell_ms20p_check','sell_underprice_check','sell_msvalue_check','buy_check','sell_check']]
    return df

In [17]:
def calculate_market_phase_raw(df):
    """
    Tính toán cột 'market_phase_raw' dựa trên các điều kiện:
      - Nếu 'buy_check' == 2         => market_phase_raw = 2
      - Nếu 'buy_check' == 1 và 'score_check' == 1 => market_phase_raw = 1
      - Nếu 'sell_check' == 1        => market_phase_raw = -1
      - Nếu 'sell_check' == 2        => market_phase_raw = -2
      - Các trường hợp khác           => market_phase_raw = NaN

    Sau đó, thực hiện backfill (bfill) cho các giá trị thiếu (NaN) và ép dòng cuối cùng về 1.
    
    Parameters:
      df (DataFrame): DataFrame chứa các cột 'buy_check', 'score_check', 'sell_check'.
    
    Returns:
      Series: cột 'market_phase_raw' sau khi tính toán và xử lý.
    """
    df_copy = df.copy()

    conditions = [
        df_copy['buy_check'] == 2,
        (df_copy['buy_check'] == 1) & (df_copy['score_check'] == 1),
        df_copy['sell_check'] == 1,
        df_copy['sell_check'] == 2
    ]
    choices = [2, 1, -1, -2]
    
    # Tạo cột market_phase_raw theo điều kiện, nếu không thỏa thì gán NaN
    df_copy['market_phase_raw'] = np.select(conditions, choices, default=np.nan)
    
    # Sử dụng backfill để lấp đầy các giá trị NaN
    df_copy['market_phase_raw'] = df_copy['market_phase_raw'].bfill()
    
    # Đảm bảo dòng cuối cùng luôn là 1 (mua)
    df_copy.loc[df_copy.index[-1], 'market_phase_raw'] = 1
    
    return df_copy['market_phase_raw']

In [18]:
def adjust_market_phase(df):
    """
    Hàm nhận vào DataFrame có cột 'market_phase_raw' (các giá trị: 1, 2, -1, -2)
    và tạo cột binary (1: mua, 0: bán) theo quy tắc kiểm tra từ dưới lên trên với “khóa” trạng thái:
    
      - Dòng cuối (thời gian mới nhất) được xác định theo dấu của market_phase_raw:
            Nếu > 0 -> 1 (mua), nếu < 0 -> 0 (bán).
      - Khi duyệt ngược từ dưới lên trên:
          + Nếu trạng thái hiện hành là mua (state > 0):
                - Nếu gặp tín hiệu âm:
                      * Nếu tín hiệu là -1: chuyển sang bán (state = -1).
                      * Nếu tín hiệu là -2: giữ mua (không cập nhật state).
                - Nếu gặp tín hiệu dương:
                      * Nếu state hiện hành là 2 và tín hiệu mới là 1, thì giữ lại state = 2.
                      * Các trường hợp khác: cập nhật state = tín hiệu mới.
          + Nếu trạng thái hiện hành là bán (state < 0):
                - Nếu gặp tín hiệu dương:
                      * Nếu tín hiệu là 1: chuyển sang mua (state = 1).
                      * Nếu tín hiệu là 2: giữ bán (không cập nhật state).
                - Nếu gặp tín hiệu âm:
                      * Nếu state hiện hành là -2 và tín hiệu mới là -1, thì giữ lại state = -2.
                      * Các trường hợp khác: cập nhật state = tín hiệu mới.
          + Sau mỗi bước, gán binary[i] = 1 nếu state > 0, ngược lại 0.
    """
    phases = df['market_phase_raw'].tolist()
    n = len(phases)
    binary = [None] * n

    # Dòng cuối cùng: xác định trạng thái theo tín hiệu của nó
    state = phases[-1]
    binary[-1] = 1 if state > 0 else 0

    # Duyệt ngược từ dòng áp chót về đầu DataFrame
    for i in range(n - 2, -1, -1):
        p = phases[i]
        if state > 0:  # Hiện đang ở trạng thái mua
            if p < 0:
                # Nếu tín hiệu âm: chỉ chuyển nếu là -1 (đảo chiều)
                if p == -1:
                    state = -1
                elif p == -2:
                    # Tín hiệu -2 không làm chuyển trạng thái, giữ mua
                    pass
            elif p > 0:
                # Nếu tín hiệu dương: kiểm tra "khóa" trạng thái mua
                # Nếu đang ở phase 2 và tín hiệu mới là 1, thì giữ phase 2 (không giảm xuống phase 1)
                if state == 2 and p == 1:
                    pass
                else:
                    state = p
            # Nếu p == 0 (không có tín hiệu) thì giữ nguyên state
        else:  # state < 0, hiện đang ở trạng thái bán
            if p > 0:
                # Nếu tín hiệu dương: chỉ chuyển nếu tín hiệu là 1 (đảo chiều)
                if p == 1:
                    state = 1
                elif p == 2:
                    # Tín hiệu 2 không làm chuyển trạng thái, giữ bán
                    pass
            elif p < 0:
                # Nếu tín hiệu âm: kiểm tra "khóa" trạng thái bán
                # Nếu đang ở phase -2 và tín hiệu mới là -1, thì giữ phase -2 (không giảm xuống -1)
                if state == -2 and p == -1:
                    pass
                else:
                    state = p
            # Nếu p == 0 thì giữ nguyên
        binary[i] = 1 if state > 0 else 0

    return binary


In [19]:
def final_market_phase_T3(df):
    # Tạo bản sao của DataFrame để không làm thay đổi dữ liệu gốc
    df_copy = df.copy()
    
    # Tạo cột market_phase_final dựa trên cột market_phase gốc
    df_copy['market_phase_final'] = df_copy['market_phase_adj']
    # Tạo cột tín hiệu T3, khởi tạo giá trị bằng 0
    df_copy['t3_signal'] = 0

    # Lặp ngược từ len(df_copy)-4 đến 0 để đảm bảo các phiên sau đã được xử lý
    for i in range(len(df_copy) - 4, -1, -1):
        current = df_copy['market_phase_final'].iloc[i]
        next_val = df_copy['market_phase_final'].iloc[i+1]
        next2 = df_copy['market_phase_final'].iloc[i+2]
        next3 = df_copy['market_phase_final'].iloc[i+3]

        # Nếu giá trị của phiên tiếp theo lớn hơn phiên sau và lớn hơn phiên hiện tại
        if (next_val > next2) and (next_val > current):
            # Cập nhật market_phase_final tại vị trí i với giá trị của phiên tiếp theo
            df_copy.at[df_copy.index[i], 'market_phase_final'] = next_val
            # Gán tín hiệu T3 bằng 2 (còn 2 phiên nữa mới bán được)
            df_copy.at[df_copy.index[i], 't3_signal'] = 2
        # Nếu giá trị của phiên thứ 2 sau lớn hơn phiên thứ 3 và phiên tiếp theo lớn hơn phiên hiện tại
        elif (next2 > next3) and (next_val > current):
            df_copy.at[df_copy.index[i], 'market_phase_final'] = next_val
            # Gán tín hiệu T3 bằng 1 (còn 1 phiên nữa mới bán được)
            df_copy.at[df_copy.index[i], 't3_signal'] = 1

    # Trả về 2 cột mới: market_phase_final và t3_signal
    return df_copy['market_phase_final']


- Tạo bảng market phase

In [20]:
#Tách bảng điểm dòng tiền thanh bảng dòng tiền âm và dương
group_positive_df = full_signed_group_df[full_signed_group_df['type']=='pos'].drop(columns=['type']).reset_index(drop=True)
group_negative_df = full_signed_group_df[full_signed_group_df['type']=='neg'].drop(columns=['type']).reset_index(drop=True)

#Tính phân bổ vốn hoá tổng chưa điều chỉnh
market_phase_df = date_series.copy()

#Tính dòng tiền vào và ra trung trung trong 5 phiên
market_phase_df[['hsA+','hsB+','hsC+','hsD+']] = group_positive_df[industry_perform_list][::-1].rolling(window=5, min_periods=1).mean()[::-1]
market_phase_df[['hsA-','hsB-','hsC-','hsD-']] = group_negative_df[industry_perform_list][::-1].rolling(window=5, min_periods=1).mean().abs()[::-1]

#Tính tỉ lệ dòng tiền vào/ra, nhân theo tỉ lệ và tạo cột score check để kiểm tra điều kiện dòng tiền
market_phase_df['hsA_score'] = ((market_phase_df['hsA+'] - market_phase_df['hsA-']) / (market_phase_df['hsA+'] + market_phase_df['hsA-'])) * 0.3
market_phase_df['hsB_score'] = (market_phase_df['hsB+'] - market_phase_df['hsB-']) / (market_phase_df['hsB+'] + market_phase_df['hsB-']) * 0.3
market_phase_df['hsC_score'] = (market_phase_df['hsC+'] - market_phase_df['hsC-']) / (market_phase_df['hsC+'] + market_phase_df['hsC-']) * 0.3
market_phase_df['hsD_score'] = (market_phase_df['hsD+'] - market_phase_df['hsD-']) / (market_phase_df['hsD+'] + market_phase_df['hsD-']) * 0.1

market_phase_df['hsC_score_S1'] = market_phase_df['hsC_score'].shift(-1).ffill()

market_phase_df['market_score'] = market_phase_df[['hsA_score','hsB_score','hsC_score','hsD_score']].sum(axis=1)
market_phase_df['score_check'] = market_phase_df.apply(lambda x: 1 if (x['market_score'] > 0.2) & (x['hsA_score'] > 0.12)  & (x['hsC_score'] > x['hsC_score_S1']) else 0, axis=1)


market_phase_df = market_phase_df.drop(columns=['hsA+', 'hsB+', 'hsC+', 'hsD+', 'hsA-', 'hsB-', 'hsC-', 'hsD-', 'hsC_score_S1'])

#Ghép bảng phase với bảng MS để thêm các cột tín hiệu từ MS
market_ms_chart_df = full_ms_chart_df[full_ms_chart_df['ticker']=='all'][['date','trend_5p','trend_20p','trend_60p']]
market_ms_chart_df = calculate_market_ms(market_ms_chart_df).sort_values('date', ascending=False).reset_index(drop=True)
market_phase_df = market_phase_df.merge(market_ms_chart_df, on='date', how='left')

#Điều chỉnh lại các giai đoạn mua bán tạo thành 2 giai đoạn chính mua (1) và bán (0)
market_phase_df['market_phase_raw'] = calculate_market_phase_raw(market_phase_df)
market_phase_df['market_phase_adj'] = adjust_market_phase(market_phase_df)
market_phase_df['market_phase_final'] = final_market_phase_T3(market_phase_df)

##### Lọc danh sách cổ phiếu từng quý

- Các hàm tính toán

In [21]:
def calculate_capital(df, return_column, initial_capital):
    """
    Hàm tính số tiền sau mỗi ngày dựa trên cột return_column (pct return) và số vốn ban đầu.
    Quy tắc tính: 
        capital[t] = initial_capital * (1 + return[t1]) * (1 + return[t2]) * ... * (1 + return[t])
    Trong đó chuỗi tính theo thứ tự thời gian từ ngày cũ nhất đến ngày hiện tại.
    
    Vì DataFrame được sắp xếp giảm dần (hàng đầu tiên là ngày hiện tại),
    ta cần đảo ngược thứ tự để tính tích lũy, sau đó gán kết quả lại cho df ban đầu.
    
    Trả về Series 'capital'.
    """
    df_copy = df.iloc[::-1].copy()
    df_copy['capital'] = initial_capital * (1 + df_copy[return_column]).cumprod()
    return df_copy['capital'][::-1]

def calculate_pct_return(df, phase_column):
    df_copy = df.copy()
    df_copy['return'] = 0
    df_copy['phase_s1'] = df_copy[phase_column].shift(-1).ffill()
    df_copy['phase_s2'] = df_copy[phase_column].shift(-2).ffill()
    df_copy['close_s1'] = df_copy['close'].shift(-1).ffill()

    df_copy['return'] = df_copy.apply(lambda x: 
        #Ngày đầu tiên có tín hiệu, ví là tín hiệu vào cuối phiên nên ko tính return
        0 if (x[phase_column]==1) & (x['phase_s1']==0) & (x['phase_s2']==0) else (
        #Ngày đầu tiên tính return, tức là ngày thứ 2 kể từ cuối phiên biết có tín hiệu
        (x['close'] - x['open'])/x['open'] if (x[phase_column]==1) & (x['phase_s1']==1) & (x['phase_s2']==0) else (
        #Các ngày giữa chu kì mua thì tính return bằng giá close như bình thường
        (x['close'] - x['close_s1'])/x['close_s1'] if (x[phase_column]==1) & (x['phase_s1']==1) & (x['phase_s2']==1) else (
        #Ngày đầu tiên có tín hiệu bán, nhưng chưa thể bán ngay nên vẫn tính như thường
        (x['close'] - x['close_s1'])/x['close_s1'] if (x[phase_column]==0) & (x['phase_s1']==1) & (x['phase_s2']==1) else (
        #Sau khi có tín hiệu bán 1 ngày, tính return dựa vào giá open vì sẽ bán ATO
        (x['open'] - x['close_s1'])/x['close_s1'] if (x[phase_column]==0) & (x['phase_s1']==0) & (x['phase_s2']==1) else 
        #Các ngày khác return lại tính bằng 0
        0)))), axis=1)
    
    return df_copy['return']

In [22]:
#Hàm tính toán hiệu suất sử dụng cho từng chổ phiếu
def calculate_ticker_perform(df):
    #Tính toán return dựa theo giá trị T2M Index
    ticker_perform_df = df.copy()
    ticker_perform_df['ticker_phase_final'] = market_phase_df['market_phase_final']
    ticker_perform_df['ticker_phase_shifted'] = ticker_perform_df['ticker_phase_final'].shift(-1).ffill()

    ticker_perform_df['ticker_return'] = calculate_pct_return(ticker_perform_df, 'ticker_phase_final')
    ticker_perform_df['ticker_cum_return'] = ticker_perform_df['ticker_return'][::-1].cumsum()[::-1]
    ticker_perform_df['ticker_capital'] = calculate_capital(ticker_perform_df, 'ticker_return', 1000)

    #Tính toán dummy return cho vnindex nếu không áp dụng bộ tín hiệu
    ticker_perform_df['dummy_phase'] = 1
    ticker_perform_df['dummy_return'] = calculate_pct_return(ticker_perform_df, 'dummy_phase')
    ticker_perform_df['dummy_cum_return'] = ticker_perform_df['dummy_return'][::-1].cumsum()[::-1]
    ticker_perform_df['dummy_capital'] = calculate_capital(ticker_perform_df, 'dummy_return', 1000)

    ticker_perform_df['dummy_return'] = ticker_perform_df['dummy_return']
    ticker_perform_df['dummy_cum_return'] = ticker_perform_df['dummy_cum_return']
    ticker_perform_df['dummy_capital'] = ticker_perform_df['dummy_capital']

    ticker_perform_df['ticker_return'] = ticker_perform_df['ticker_return']
    ticker_perform_df['ticker_cum_return'] = ticker_perform_df['ticker_cum_return']
    ticker_perform_df['ticker_capital'] = ticker_perform_df['ticker_capital']

    return ticker_perform_df

In [23]:
#Hàm cắt stock dict ra thành từng quý, sau đó tính toán performance của từng quý để lọc cổ phiếu có performance thấp
def create_quarter_stock_dict_for_signal(stock_dict, stock_classification_df):
    # Convert to set for faster lookups
    signal_stock_set = set(stock_classification_df[stock_classification_df['industry_perform'].isin(['hsA', 'hsB', 'hsC'])]['ticker'])

    #Tạo danh sách cổ phiếu cho từng quý
    signal_quarter_stock_df_dict = {}

    #Tạo các cặp quý liên tiếp để chạy vòng lặp - using list comprehension for efficiency
    quarter_pairs = [(quarter_name_list[0], quarter_name_list[0])]
    quarter_pairs.extend([(quarter_name_list[i], quarter_name_list[i + 1]) for i in range(len(quarter_name_list) - 1)])
    
    # Precompute sets for faster filtering
    total_stock_set = set(total_stock_list)
    quarter_stock_sets = {q: set(quarter_stock_list[q].dropna()) for q in quarter_name_list}

    #Chạy vòng lặp qua từng quý
    for prev_quarter, current_quarter in quarter_pairs:
        # Get date range once - fixing the unpacking issue
        start_date, end_date = quarter_map_dict[prev_quarter][0], quarter_map_dict[prev_quarter][1]
        
        # Pre-filter relevant tickers using set operations (much faster)
        relevant_tickers = quarter_stock_sets[current_quarter] & total_stock_set & signal_stock_set
        
        # Collect performance data in a list rather than concatenating DataFrames
        performance_data = []
        for ticker in relevant_tickers:
            if ticker in stock_dict:
                df = stock_dict[ticker]
                temp_df = df[(df['date'] >= start_date) & (df['date'] <= end_date)].reset_index(drop=True)
                if not temp_df.empty:
                    perf_df = calculate_ticker_perform(temp_df)
                    if not perf_df.empty:
                        perf_row = perf_df.iloc[0].copy()
                        perf_row.name = ticker  # Preserve ticker as row name
                        performance_data.append(perf_row)
        
        # Create DataFrame in one operation instead of concatenating in a loop
        signal_quarter_stock_df_dict[current_quarter] = pd.DataFrame(performance_data) if performance_data else pd.DataFrame()

    return signal_quarter_stock_df_dict

In [24]:
#Hàm lọc cổ phiếu theo quý dựa trên hiệu suất với quý trước và gộp vào cùng danh sách cổ phiếu bắt buộc
def filter_quarter_stock_list_dict(signal_quarter_stock_dict, mandatory_stock_list, prohibited_stock_list):
    temp_quarter_stock_list_dict = {}
    # Convert prohibited list to a set once for efficiency
    prohibited_set = set(prohibited_stock_list)

    for quarter in quarter_name_list:
        # Tính index của quý hiện tại
        current_quarter_index = quarter_name_list.index(quarter)

        # Khởi tạo giá trị của calculate_quarter_index và mean_cum_return
        calculate_quarter_index = current_quarter_index
        mean_cum_return = signal_quarter_stock_dict[quarter_name_list[calculate_quarter_index]]['ticker_cum_return'].mean()

        # Tạo vòng lặp để kiểm tra và lùi quý tới khi nào tìm đc quý có mean_cum_return > 0
        while mean_cum_return < 0:
            calculate_quarter_index = calculate_quarter_index - 1
            mean_cum_return = signal_quarter_stock_dict[quarter_name_list[calculate_quarter_index]]['ticker_cum_return'].mean()

        temp_quarter_perform_df = signal_quarter_stock_dict[quarter_name_list[calculate_quarter_index]].copy()
        
        # Step 1: Get stocks passing performance filter as a set
        performance_stocks = set(temp_quarter_perform_df[temp_quarter_perform_df['ticker_cum_return'] > mean_cum_return * 2]['ticker'])
        
        # Step 2: Add mandatory stocks
        candidate_stocks = performance_stocks.union(mandatory_stock_list)
        
        # Step 3: Get available stocks in current quarter as a set
        available_stocks = set(quarter_stock_df[quarter_name_list[current_quarter_index]].dropna())
        
        # Step 4: Final filtering - keep stocks that are candidates AND available BUT NOT prohibited
        final_stocks = candidate_stocks.intersection(available_stocks).difference(prohibited_set)
        
        # Convert to sorted list for consistent output
        temp_quarter_stock_list_dict[quarter_name_list[current_quarter_index]] = sorted(list(final_stocks))

    return temp_quarter_stock_list_dict

In [25]:
#Hàm tính toán trung bình thay đổi của tất cả cổ phiếu trong danh sách
def calculate_total_change(stock_group, price_index_date_series):
    quarter_index_df = price_index_date_series.copy()

    for ticker, df in stock_group.items():
        quarter_index_df[ticker] = df['open']
        quarter_index_df[ticker] = quarter_index_df[ticker][::-1].pct_change()[::-1]

    quarter_index_df['total_change'] = quarter_index_df.iloc[:,1:].mean(axis=1)
    quarter_index_df = quarter_index_df.fillna(0)

    return quarter_index_df

- Tạo dict chứa danh sách cổ phiếu từng quý sau khi lọc

In [26]:
#Danh sách cổ phiếu bắt buộc phải có trong bộ tín hiệu
mandatory_stock_list = stock_classification_df[stock_classification_df['signal']=='o']['ticker'].tolist()
prohibited_stock_list  = stock_classification_df[stock_classification_df['signal']=='x']['ticker'].tolist()

#Hàm cắt stock dict ra thành từng quý, sau đó tính toán performance quý cho từng cổ phiếu
signal_quarter_stock_df_dict = create_quarter_stock_dict_for_signal(stock_dict, stock_classification_df)

#Tạo dict cổ phiếu sau khi lọc qua điều kiện performance và kết hợp với danh sách cổ phiếu bắt buộc
signal_quarter_stock_list_dict = filter_quarter_stock_list_dict(signal_quarter_stock_df_dict, mandatory_stock_list, prohibited_stock_list)

- Tạo các bảng tính toán biến động trung bình toàn bộ cổ phiếu

In [27]:
#Chuyển dict trên từ dạng list sang dạng df để tiến hành tính toán hiệu suất
quarter_stock_dict_filtered = {}
for quarter in quarter_name_list:
    quarter_stock_dict_filtered[quarter] = {key: value for key, value in quarter_stock_dict[quarter].items() if key in signal_quarter_stock_list_dict[quarter]}

#Dict tính tổng biến động của các cổ phiếu chưa lọc và đã lọc trong từng giai đoa
market_change_dict = {}
filtered_change_dict = {}
for quarter in quarter_name_list:
    market_change_dict[quarter] = calculate_total_change(quarter_stock_dict[quarter], quarter_date_series_dict[quarter])
    filtered_change_dict[quarter] = calculate_total_change(quarter_stock_dict_filtered[quarter], quarter_date_series_dict[quarter])

market_change_df = pd.DataFrame()
filtered_change_df = pd.DataFrame()
for quarter in quarter_name_list:
    market_change_df = pd.concat([market_change_df, market_change_dict[quarter][['date', 'total_change']]], axis=0)
    filtered_change_df = pd.concat([filtered_change_df, filtered_change_dict[quarter][['date', 'total_change']]], axis=0)

market_change_df = market_change_df.sort_values('date', ascending=False).reset_index(drop=True)
filtered_change_df = filtered_change_df.sort_values('date', ascending=False).reset_index(drop=True)

##### Tạo bảng market perform 

In [28]:
#Tạo bảng vnindex và tính toán return dựa theo vnindex
vnindex_df = full_index_df[full_index_df['ticker']=='VNINDEX'].reset_index(drop=True)

#Tạo bảng để so sánh hiệu suất đầu tư
market_perform_df = date_series.copy()
market_perform_df[['trend_5p','trend_20p','trend_60p']] = market_ms_chart_df[['trend_5p','trend_20p','trend_60p']]
market_perform_df[['open','high','low','close']] = vnindex_df[['open','high','low','close']]
market_perform_df['market_phase_final'] = market_phase_df['market_phase_final']

#Tính toán dummy return cho vnindex nếu không áp dụng bộ tín hiệu
market_perform_df['dummy_phase'] = 1
market_perform_df['dummy_return'] = calculate_pct_return(market_perform_df, 'dummy_phase')
market_perform_df['vnindex_return'] = calculate_pct_return(market_perform_df, 'market_phase_final')

#### Bộ phân bổ vốn tự động

##### Các hàm tính toán

In [29]:
#Hàm này trả về các nhóm hsA hoặc hsBC để tính toán trong các hàm dưới
def calculate_group_industry_perform(industry_perform):
    if industry_perform == 'hsA':
        return 'hsA'
    elif (industry_perform == 'hsB') | (industry_perform == 'hsC'):
        return 'hsBC'

#Hàm này tính toán tỉ trọng mỗi khi pick cổ phiếu
def calculate_stock_portion(prev_pct_change, group_industry_perform, temp_portfolio_stock, number_of_stock):
    #Điều chỉnh tỉ trọng tăng giảm
    if len(temp_portfolio_stock) != (number_of_stock - 1):
        if number_of_stock == 6:
            if group_industry_perform == 'hsA':
                return 13/75 * (1 + prev_pct_change/2)
            elif group_industry_perform == 'hsBC':
                return 12/75 * (1 + prev_pct_change/2)
        elif number_of_stock == 10:
            if group_industry_perform == 'hsA':
                return 0.12 * (1 + prev_pct_change/2)
            elif group_industry_perform == 'hsBC':
                return 0.08 * (1 + prev_pct_change/2)
    elif len(temp_portfolio_stock) == (number_of_stock - 1):
        current_total_portion = sum(0 if stock_info.get('portion') is None else 
                                    stock_info.get('portion', 0) for stock_info in temp_portfolio_stock.values())
        return 1 - current_total_portion

#Hàm này để đếm số lượng cổ phiéu đang nắm giữ ở từng nhóm
def stock_count_portfolio(portfolio_count_dict):
    # Đếm số lượng trong hsA
    hsA_dict = portfolio_count_dict.get('hsA', {})
    hsA_count = sum(hsA_dict.values())
    
    # Đếm số lượng trong hsBC
    hsBC_dict = portfolio_count_dict.get('hsBC', {})
    hsBC_count = sum(hsBC_dict.values())
    
    # Tổng số lượng trong toàn dict
    total_count = hsA_count + hsBC_count
    
    return {'total': total_count, 'hsA': hsA_count, 'hsBC': hsBC_count}

#Hàm này để cập nhật temp_portfolio_count cho ngày mới bằng những cổ phiếu đang nắm giữ từ hôm trước
def update_portfolio_portion_count(temp_portfolio_stock):
    temp_portfolio_count = {'hsA': {'bds': 0,'xd': 0,'chung_khoan': 0,'tai_chinh': 0,'thep': 0,'ban_le': 0,},
                            'hsBC': {'thuy_san': 0,'det_may': 0,'cong_nghiep': 0,'hoa_chat': 0,'khoang_san': 0,'dau_khi': 0,
                                    'bds_kcn': 0,'cong_nghe': 0,'ngan_hang': 0,'van_tai': 0,'thuc_pham': 0,'htd': 0,}}
    if len(temp_portfolio_stock) > 0:
        for item in temp_portfolio_stock.values():
            if item['industry_name'] in temp_portfolio_count['hsA'].keys():
                temp_portfolio_count['hsA'][item['industry_name']] += 1
            else:
                temp_portfolio_count['hsBC'][item['industry_name']] += 1 
                
    return temp_portfolio_count

#Hàm này chọn cổ phiếu để mua mới
def picking_stock(df, temp_portfolio_count, number_of_total_stock, number_of_group_stock, temp_portfolio_stock_count, classification_df):
    df = df.sort_values('t5_score', ascending=False)
    buy_stock_list = []

    for ticker in df['ticker'].tolist():
        # Check điều kiện xem tổng số lượng cổ phiếu đã đủ chưa
        if stock_count_portfolio(temp_portfolio_count)['total'] < number_of_total_stock:

            temp_industry_name = classification_df.set_index('ticker').loc[ticker, 'industry_name']
            temp_industry_perform = classification_df.set_index('ticker').loc[ticker, 'industry_perform']
            group_industry_perform = calculate_group_industry_perform(temp_industry_perform)

            # Check điều kiện xem nhóm hiêu suất đã đủ cổ phiếu chưa, nếu đủ rồi thì bỏ qua cổ phiếu này
            if not stock_count_portfolio(temp_portfolio_count)[group_industry_perform] \
                < number_of_group_stock[group_industry_perform]:
                    continue

            # Check điều kiện xem ngành đã đủ cổ phiếu chưa, nếu đủ rồi thì bỏ qua cổ phiếu này
            if not temp_portfolio_count[group_industry_perform][temp_industry_name] \
                < temp_portfolio_stock_count[group_industry_perform][temp_industry_name]:
                    continue

            #Thêm các cổ phiếu mua với vào list để lưu vào cột buy signal
            buy_stock_list.append(ticker)

            #Tăng các biến đếm số lượng cổ phiếu
            temp_portfolio_count[group_industry_perform][temp_industry_name] += 1

    return buy_stock_list


In [30]:
def technical_filter_df(df, date_key, stock_dict, raw_stock_list, marketcap_template):

    # Ngưỡng xác định xu hướng thị trường
    index_week_threshold = -0.01  # Ngưỡng chỉ số tuần
    index_month_threshold = -0.02  # Ngưỡng chỉ số tháng

    # Cấu hình giá trị vốn hóa tối thiểu cho từng loại
    marketcap_template_dict = {
        'small': {'min_value': 500000,'mean_value': 500000},
        'medium': {'min_value': 5000000,'mean_value': 5000000},
        'large': {'min_value': 10000000,'mean_value': 10000000}   
    }

    # Thu thập dữ liệu cổ phiếu tại ngày giao dịch cần kiểm tra
    frames = []
    for ticker in raw_stock_list:
        stock_data = stock_dict[ticker]
        day_data = stock_data[stock_data['date'] == date_key]
        if not day_data.empty:
            frames.append(day_data.copy())
            
    # Kiểm tra nếu không có dữ liệu nào
    if not frames:
        return pd.DataFrame()
    
    # Gộp tất cả dữ liệu cổ phiếu vào một DataFrame
    raw_stock_df = pd.concat(frames, ignore_index=True)

    # Lọc cổ phiếu theo điều kiện thanh khoản và vốn hóa
    template_limits = marketcap_template_dict[marketcap_template]
    filtered_stock_df = raw_stock_df[
        (raw_stock_df['t5_score'] > -1) &
        # Thanh khoản tối thiểu và trung bình phải > 50,000
        (raw_stock_df['min_vol'] > 50000) &
        (raw_stock_df['mean_vol'] > 50000) &
        # Vốn hóa phải lớn hơn ngưỡng theo loại
        (raw_stock_df['min_value'] > template_limits['min_value']) & 
        (raw_stock_df['mean_value'] > template_limits['mean_value'])
    ]

    # Lấy thông tin xu hướng thị trường tại ngày giao dịch đang xét
    market_info = df.loc[df['date'] == date_key].iloc[0]
    trend_5p = market_info['trend_5p']    # Xu hướng ngắn hạn (5 phiên)
    trend_20p = market_info['trend_20p']  # Xu hướng trung hạn (20 phiên)

    # Điều kiện biến động giá (diff_10 phải nằm trong khoảng 0% đến 10%)
    price_change_condition = (filtered_stock_df['diff_lowest10'] < 0.1) & (filtered_stock_df['diff_lowest10'] > 0)
    
    # Áp dụng bộ lọc khác nhau tùy theo loại vốn hóa và tình trạng thị trường
    if marketcap_template == 'small':  # Cổ phiếu vốn hóa nhỏ
        if (trend_5p > 0.9 or trend_20p > 0.8) or (trend_5p < 0.2 or trend_20p < 0.2):
            filtered_stock_df = filtered_stock_df[price_change_condition]
    elif marketcap_template == 'medium':  # Cổ phiếu vốn hóa trung bình
        if (trend_5p > 0.7 and trend_20p > 0.7) or (trend_5p < 0.4 and trend_20p < 0.3):
            filtered_stock_df = filtered_stock_df[price_change_condition]
    elif marketcap_template == 'large':  # Cổ phiếu vốn hóa lớn
        if (trend_5p > 0.8 and trend_20p > 0.8) or (trend_5p < 0.3 and trend_20p < 0.2):
            filtered_stock_df = filtered_stock_df[price_change_condition]

    # Kiểm tra xu hướng thị trường để áp dụng lọc theo ngưỡng Fibonacci
    # Nếu thị trường không giảm quá nhiều trong tuần (5 phiên)
    if market_info['return_cumsum5'] > index_week_threshold:
        # Giá đóng cửa phải cao hơn mức Fibonacci 61.8% theo tuần
        filtered_stock_df = filtered_stock_df[filtered_stock_df['close'] > filtered_stock_df['WFIBO_0618']]
    
    # Nếu thị trường không giảm quá nhiều trong tháng (20 phiên)
    if market_info['return_cumsum20'] > index_month_threshold:
        # Giá đóng cửa phải cao hơn mức Fibonacci 61.8% theo tháng
        filtered_stock_df = filtered_stock_df[filtered_stock_df['close'] > filtered_stock_df['MFIBO_0618']]

    return filtered_stock_df

In [31]:
def selling_stock(temp_portfolio_stock, marketcap_template, date_key, stock_dict):
    sell_stock_df = pd.DataFrame()
    for ticker in temp_portfolio_stock.keys():
        temp_df = stock_dict[ticker].copy()
        temp_df = temp_df[temp_df['date'] == date_key]
            
        if marketcap_template == 'small':
            temp_df = temp_df[
                    (temp_df['diff_highest10'] < -0.10) |
                    ((temp_df['t5_score'] < -0.1) & 
                        (temp_df['close'] < temp_df['WFIBO_0382']) & 
                        (temp_df['close'] > temp_df['WFIBO_0618'])
            )]
        elif marketcap_template == 'medium':
            temp_df = temp_df[
                    ((temp_df['t5_score'] < -0.1) & 
                        (temp_df['close'] < temp_df['WFIBO_0382']) & 
                        (temp_df['close'] > temp_df['WFIBO_0618']) &
                        (temp_df['diff_lowest1'] < 0.03)
            )]
        elif marketcap_template == 'large':
            temp_df = temp_df[
                    ((temp_df['t5_score'] < -0.1) & 
                        (temp_df['close'] < temp_df['WFIBO_0382']) & 
                        (temp_df['close'] > temp_df['WFIBO_0618']) 
            )]              
                    
        sell_stock_df = pd.concat([sell_stock_df, temp_df], axis=0)

    # Chỉ bán các cổ phiếu đã nắm giữ ít nhất 2 ngày (đủ điều kiện T+)
    sell_stock_list = []
    if len(sell_stock_df) > 0:
        for ticker in sell_stock_df['ticker'].tolist():
            if temp_portfolio_stock[ticker]['days_held'] >= 2:
                sell_stock_list.append(ticker)

    return sell_stock_list

In [32]:
def auto_portfolio(df, stock_dict, classification_df, portfolio_template, marketcap_template):
    """
    Hàm tự động xây dựng và quản lý danh mục đầu tư theo thời gian
    
    Args:
        df: DataFrame chỉ số thị trường chứa các tín hiệu giao dịch
        stock_dict: Dictionary chứa dữ liệu cổ phiếu theo mã
        classification_df: DataFrame phân loại ngành nghề của các cổ phiếu
        portfolio_template: Loại danh mục ('6stocks', '10stocks', '20stocks')
        marketcap_template: Loại vốn hóa ('small', 'medium', 'large')
    
    Returns:
        df: DataFrame đã cập nhật tín hiệu mua/bán
        holding_stock_dict: Dictionary theo dõi các cổ phiếu nắm giữ theo thời gian
    """
    df = df.copy()

    # Cấu hình số lượng cổ phiếu phân bổ theo ngành cho từng loại danh mục
    portfolio_stock_count_dict = {
        '6stocks': {
            'hsA': {  # Nhóm ngành tăng trưởng cao
                'bds': 2, 'xd': 2,'chung_khoan': 2,'tai_chinh': 1,'thep': 1,'ban_le': 1,
            },
            'hsBC': {  # Nhóm ngành tăng trưởng thấp/trung bình
                'thuy_san': 1, 'det_may': 1, 'cong_nghiep': 1, 'hoa_chat': 1,
                'khoang_san': 1, 'dau_khi': 1, 'bds_kcn': 1, 'cong_nghe': 1,
                'ngan_hang': 1, 'van_tai': 1, 'thuc_pham': 1, 'htd': 1,
            }
        },
        '10stocks': {
            'hsA': {
                'bds': 3, 'xd': 2, 'chung_khoan': 3, 'tai_chinh': 1, 'thep': 2, 'ban_le': 1,
            },
            'hsBC': {
                'thuy_san': 1, 'det_may': 1, 'cong_nghiep': 2, 'hoa_chat': 2, 'khoang_san': 1,
                'dau_khi': 1, 'bds_kcn': 2, 'cong_nghe': 2, 'ngan_hang': 2, 'van_tai': 2,
                'thuc_pham': 2, 'htd': 1,
            }
        },
    }


    # Lấy cấu hình cho loại danh mục đã chọn
    temp_portfolio_stock_count = portfolio_stock_count_dict[portfolio_template]

    # Thiết lập số lượng cổ phiếu tối đa và phân bổ theo nhóm
    if portfolio_template == '6stocks':
        number_of_total_stock = 6
        number_of_group_stock = {'hsA': 3, 'hsBC': 3}
    elif portfolio_template == '10stocks':
        number_of_total_stock = 10
        number_of_group_stock = {'hsA': 5, 'hsBC': 5}

    # Dictionary lưu trữ các cổ phiếu nắm giữ theo từng ngày
    holding_stock_dict = {}

    # Duyệt từ cuối lên đầu danh sách để xây dựng danh mục theo thời gian
    for i in range(len(df) - 1, -1, -1):
        date_key = df.at[i, 'date'].strftime('%Y-%m-%d')  # Ngày hiện tại

        # --- XỬ LÝ NGÀY ĐẦU TIÊN ---
        if i == len(df) - 1:
            # Nếu ngày đầu tiên có tín hiệu mua mới (phase=1, phase_s1=0, phase_s2=0)
            if ((df.at[i, 'phase'] == 1) & (df.at[i, 'phase_s1'] == 0) & (df.at[i,'phase_s2'] == 0)):
                # Khởi tạo danh mục trống
                temp_portfolio_count = update_portfolio_portion_count({})
                raw_stock_list = df.at[i, 'raw_holding_stock']
                # Lọc cổ phiếu theo tiêu chí kỹ thuật
                filtered_stock_df = technical_filter_df(df, date_key, stock_dict, raw_stock_list, marketcap_template)
                # Chọn cổ phiếu để mua
                df.at[i, 'buy_signal'] = picking_stock(filtered_stock_df, temp_portfolio_count, 
                                                      number_of_total_stock, number_of_group_stock, 
                                                      temp_portfolio_stock_count, classification_df)

            # Ngày đầu tiên chưa có cổ phiếu nắm giữ
            holding_stock_dict[date_key] = {}
            continue

        # --- XỬ LÝ CÁC NGÀY TIẾP THEO ---
        # Lấy thông tin ngày trước đó
        prev_date_key = df.at[i+1, 'date'].strftime('%Y-%m-%d')
        
        # Lọc và cập nhật danh mục từ ngày trước (loại bỏ các cổ phiếu đã có tín hiệu bán)
        temp_portfolio_stock = {
            key: copy.deepcopy(value) 
            for key, value in holding_stock_dict[prev_date_key].items()
            if key not in df.at[i+1, 'sell_signal']
        }
        
        # Tăng số ngày nắm giữ cho các cổ phiếu còn lại
        for ticker in temp_portfolio_stock.keys():
            temp_portfolio_stock[ticker]['days_held'] += 1

        # --- THÊM CỔ PHIẾU MỚI VÀO DANH MỤC ---
        # Xử lý các cổ phiếu đã có tín hiệu mua ở phiên trước
        for ticker in df.at[i+1, 'buy_signal']:
            # Lấy thông tin ngành nghề của cổ phiếu
            temp_industry_name = classification_df.set_index('ticker').loc[ticker, 'industry_name']
            temp_industry_perform = classification_df.set_index('ticker').loc[ticker, 'industry_perform']
            group_industry_perform = calculate_group_industry_perform(temp_industry_perform)

            # Lấy giá mở cửa và giá tham chiếu
            open_price = stock_dict[ticker].set_index('date').loc[date_key, 'open']
            prev_close_price = stock_dict[ticker].set_index('date').loc[prev_date_key, 'close']
            prev_pct_change = stock_dict[ticker].set_index('date').loc[prev_date_key, 'pct_change']

            # Chỉ mua cổ phiếu khi giá mở cửa không tăng quá 2% so với giá tham chiếu
            if (((open_price - prev_close_price)/prev_close_price) < 0.02):
                temp_portfolio_stock[ticker] = {
                    'buy_price': open_price,
                    'industry_name': temp_industry_name,
                    'industry_perform': temp_industry_perform,
                    'days_held': 0,  # Mới mua nên days_held = 0
                    'portion': calculate_stock_portion(prev_pct_change, group_industry_perform, temp_portfolio_stock, number_of_total_stock)
                }
        
        # --- CẬP NHẬT THÔNG TIN DANH MỤC HIỆN TẠI ---
        # Tính toán số lượng cổ phiếu theo nhóm ngành
        temp_portfolio_count = update_portfolio_portion_count(temp_portfolio_stock)
        # Lưu danh mục ngày hiện tại
        holding_stock_dict[date_key] = copy.deepcopy(temp_portfolio_stock)
        # Cập nhật danh sách cổ phiếu đang nắm giữ vào DataFrame
        df.at[i, 'filtered_holding_stock'] = list(temp_portfolio_stock.keys())

        # --- LỌC CỔ PHIẾU TIỀM NĂNG CHO PHIÊN HIỆN TẠI ---
        raw_stock_list = df.at[i, 'raw_holding_stock']
        filtered_stock_df = technical_filter_df(df, date_key, stock_dict, raw_stock_list, marketcap_template)

        # Loại bỏ các cổ phiếu đã nắm giữ hoặc đã có tín hiệu mua
        if len(filtered_stock_df) > 0: 
            filtered_stock_df = filtered_stock_df[~filtered_stock_df['ticker'].isin(holding_stock_dict[prev_date_key].keys())]
            filtered_stock_df = filtered_stock_df[~filtered_stock_df['ticker'].isin(df.at[i+1, 'buy_signal'])]

        # --- XỬ LÝ THEO TỪNG GIAI ĐOẠN CỦA CHU KỲ GIAO DỊCH ---
        # Ngày đầu tiên của chu kỳ mới (phase=1, phase_s1=0, phase_s2=0)
        if ((df.at[i, 'phase'] == 1) & (df.at[i, 'phase_s1'] == 0) & (df.at[i,'phase_s2'] == 0)):
            # Chọn cổ phiếu mua mới
            df.at[i, 'buy_signal'] = picking_stock(filtered_stock_df, temp_portfolio_count, 
                                                 number_of_total_stock, number_of_group_stock, 
                                                 temp_portfolio_stock_count, classification_df)
        
        # Ngày thứ hai của chu kỳ mới (phase=1, phase_s1=1, phase_s2=0)
        elif ((df.at[i, 'phase'] == 1) & (df.at[i, 'phase_s1'] == 1) & (df.at[i,'phase_s2'] == 0)):
            # Không cần làm gì, chỉ mua cổ phiếu theo tín hiệu phiên trước
            pass

        # Giai đoạn chính của chu kỳ giao dịch (phase=1, phase_s1=1, phase_s2=1)
        elif ((df.at[i, 'phase'] == 1) & (df.at[i, 'phase_s1'] == 1) & (df.at[i,'phase_s2'] == 1)):                
            # Cập nhật tín hiệu bán
            df.at[i, 'sell_signal'] = selling_stock(temp_portfolio_stock, marketcap_template, date_key, stock_dict)
            
            # Chọn cổ phiếu mới để thay thế các mã đã bán
            df.at[i, 'buy_signal'] = picking_stock(filtered_stock_df, temp_portfolio_count, 
                                                 number_of_total_stock, number_of_group_stock, 
                                                 temp_portfolio_stock_count, classification_df)
        # Các giai đoạn khác (tín hiệu bán hoặc kết thúc chu kỳ)
        else:
            sell_stock_list = []
            for ticker in temp_portfolio_stock.keys():
                if ticker in holding_stock_dict[date_key].keys():
                    if temp_portfolio_stock[ticker]['days_held'] >= 2:
                        sell_stock_list.append(ticker)

            df.at[i, 'sell_signal'] = list(sell_stock_list)
        
    # Tính số lượng cổ phiếu trong danh mục theo ngày
    df['number_of_stock'] = df['filtered_holding_stock'].apply(lambda x: len(x))
    
    return df, holding_stock_dict

In [33]:
def calculate_stock_performance(portfolio_df, holding_dict, stock_dict):
    """
    Tính hiệu suất của các cổ phiếu trong danh mục đầu tư với tối ưu tốc độ.
    
    Args:
        portfolio_df: DataFrame danh mục đầu tư
        holding_dict: Dictionary chứa cổ phiếu nắm giữ theo ngày
        stock_dict: Dictionary chứa dữ liệu gốc của cổ phiếu
        date_series: Series ngày (tùy chọn, sẽ tạo từ portfolio_df nếu không cung cấp)
    
    Returns:
        Dictionary chứa DataFrame hiệu suất cho từng mã cổ phiếu
    """
    
    # Bước 1: Xác định tất cả cổ phiếu từng nằm trong danh mục để tránh lặp lại
    all_portfolio_stocks = set()
    for stocks in portfolio_df['filtered_holding_stock']:
        all_portfolio_stocks.update(stocks)
    all_portfolio_stocks = sorted(list(all_portfolio_stocks))
    
    # Bước 2: Tạo DataFrame theo dõi tỷ trọng cổ phiếu theo ngày
    stock_phase_df = portfolio_df[['date']]
    date_keys = [date.strftime('%Y-%m-%d') for date in stock_phase_df['date']]
    
    # Tạo dict ánh xạ từ date_key sang index để tra cứu nhanh
    date_to_idx = {date_key: idx for idx, date_key in enumerate(date_keys)}
    
    # Bước 3: Tạo ndarray cho tỷ trọng - nhanh hơn việc cập nhật DataFrame
    num_dates = len(stock_phase_df)
    portion_matrix = np.zeros((num_dates, len(all_portfolio_stocks)))
    
    # Lưu index của mỗi ticker để truy cập nhanh vào ma trận
    ticker_to_col_idx = {ticker: idx for idx, ticker in enumerate(all_portfolio_stocks)}
    
    # Điền tỷ trọng vào ma trận
    for date_key, holdings in holding_dict.items():
        if date_key in date_to_idx:
            row_idx = date_to_idx[date_key]
            for ticker, info in holdings.items():
                if ticker in ticker_to_col_idx:
                    col_idx = ticker_to_col_idx[ticker]
                    portion_matrix[row_idx, col_idx] = info['portion']
    
    # Bước 4: Chuyển ma trận tỷ trọng vào DataFrame
    for i, ticker in enumerate(all_portfolio_stocks):
        stock_phase_df[ticker] = portion_matrix[:, i]
    
    # Bước 5: Tính toán hiệu suất cho từng mã cổ phiếu
    stock_perform_dict = {}
    
    # Tối ưu loop chính - xử lý từng cổ phiếu
    for ticker, df in stock_dict.items():
        # Chỉ trích xuất các cột cần thiết
        temp_df = df[['date', 'open', 'close','trade_price']].copy()
        
        # Thêm cột portion
        if ticker in all_portfolio_stocks:
            temp_df['portion'] = stock_phase_df[ticker].values  # Sử dụng .values để tránh index alignment
        else:
            temp_df['portion'] = 0
        
        # Tính phase và phase_s1
        temp_df['phase'] = (temp_df['portion'] > 0).astype(int)
        temp_df['phase_s1'] = np.pad(temp_df['phase'].values[1:], (0, 1), 'constant')
        
        # Chuẩn bị arrays cho tính toán return nhanh
        n = len(temp_df)
        returns = np.zeros(n)
        
        # Lấy các arrays để tính toán vector hóa khi có thể
        phase = temp_df['phase'].values
        phase_s1 = temp_df['phase_s1'].values
        opens = temp_df['open'].values
        closes = temp_df['close'].values
        trade_prices = temp_df['trade_price'].values

        for i in range(n-2, -1, -1):
            if phase[i] == 1 and phase_s1[i] == 0:
                # Trường hợp mua
                returns[i] = (closes[i] - trade_prices[i]) / trade_prices[i]
            elif phase[i] == 1 and phase_s1[i] == 1:
                # Trường hợp tiếp tục nắm giữ
                returns[i] = (closes[i] - closes[i+1]) / closes[i+1]
            elif phase[i] == 0 and phase_s1[i] == 1:
                # Trường hợp bán
                returns[i] = (trade_prices[i] - closes[i+1]) / closes[i+1]
            # Else returns[i] đã là 0 rồi
        
        # Gán kết quả trở lại DataFrame
        temp_df['return'] = returns
        temp_df['portion_return'] = temp_df['return'] * temp_df['portion']
        
        stock_perform_dict[ticker] = temp_df
    
    return stock_perform_dict

In [34]:
def extract_trade_history(stock_perform_dict, portfolio_name):
    """
    Trích xuất lịch sử giao dịch từ dictionary hiệu suất cổ phiếu.
    
    Args:
        stock_perform_dict: Dictionary chứa DataFrame hiệu suất cho mỗi mã cổ phiếu
        portfolio_name: Tên của danh mục đầu tư
        
    Returns:
        DataFrame chứa lịch sử giao dịch với các cột: ticker, buy_date, sell_date, buy_price, sell_price, return
    """
    # Khởi tạo DataFrame rỗng để lưu kết quả
    trade_history_df = pd.DataFrame()

    # Duyệt qua từng cổ phiếu trong dictionary
    for ticker, df in stock_perform_dict.items():
        # Tạo bản sao để không ảnh hưởng dữ liệu gốc
        temp_df = df.copy().reset_index(drop=True)
        
        # Khởi tạo danh sách lưu các chỉ số của ngày mua và bán
        trading_pairs = []
        
        # Đánh dấu bắt đầu của một giai đoạn mua
        start_idx = None
        
        # Duyệt từ dưới lên (từ cũ đến mới) theo đúng thứ tự thời gian
        for i in range(len(temp_df)-1, -1, -1):
            # Tìm điểm bắt đầu mua (chuyển từ 0 sang 1)
            if i < len(temp_df)-1 and temp_df['phase'].iloc[i+1] == 0 and temp_df['phase'].iloc[i] == 1:
                start_idx = i
            
            # Tìm điểm kết thúc mua (chuyển từ 1 sang 0) và đã có điểm bắt đầu
            elif start_idx is not None and i < len(temp_df)-1 and temp_df['phase'].iloc[i+1] == 1 and temp_df['phase'].iloc[i] == 0:
                end_idx = i
                # Lưu cặp mua-bán vào danh sách với thứ tự đúng
                trading_pairs.append((start_idx, end_idx))
                start_idx = None
        
        # Trường hợp đặc biệt: nếu phase kết thúc với 1 mà chưa có điểm bán
        if start_idx is not None and temp_df['phase'].iloc[0] == 1:
            trading_pairs.append((start_idx, 0))
        
        # Tạo DataFrame mới để lưu thông tin giao dịch
        if trading_pairs:
            trade_data = {
                'buy_date': [temp_df.at[buy_idx, 'date'] for buy_idx, _ in trading_pairs],
                'sell_date': [temp_df.at[sell_idx, 'date'] for _, sell_idx in trading_pairs],
                'buy_price': [temp_df.at[buy_idx, 'trade_price'] for buy_idx, _ in trading_pairs],
                'sell_price': [temp_df.at[sell_idx, 'trade_price'] for _, sell_idx in trading_pairs],
                'portion': [temp_df.at[buy_idx, 'portion'] for buy_idx, _ in trading_pairs]
            }
            
            ticker_df = pd.DataFrame(trade_data)
            ticker_df['ticker'] = ticker
            ticker_df['return'] = (ticker_df['sell_price'] - ticker_df['buy_price']) / ticker_df['buy_price']
            ticker_df['name'] = portfolio_name
            
            trade_history_df = pd.concat([trade_history_df, ticker_df], axis=0)
    
    return trade_history_df

In [35]:
def calculate_stock_mean_return(trade_history_df, portfolio_name):
    """
    Tóm tắt hiệu suất giao dịch cho từng mã cổ phiếu
    
    Args:
        trade_history_df: DataFrame chứa lịch sử giao dịch
        
    Returns:
        DataFrame với thông tin hiệu suất trung bình và số lượt mua cho mỗi mã
    """
    if trade_history_df.empty:
        return pd.DataFrame(columns=['ticker', 'mean_return', 'buy_count'])
    
    # Sử dụng groupby để tính toán hiệu quả hơn
    result_df = trade_history_df.groupby('ticker').agg(
        mean_return=('return', 'mean'),
        buy_count=('return', 'count')
    ).reset_index()

    result_df['name'] = portfolio_name
    
    return result_df

In [36]:
def calculate_portfolio_return(date_series, stock_perform_dict, start_calculate_date, end_calculate_date):
    portfolio_return_df = date_series.copy()
    portfolio_return_df = portfolio_return_df[(portfolio_return_df['date'] >= start_calculate_date) & (portfolio_return_df['date'] <= end_calculate_date)].reset_index(drop=True)
    for ticker, df in stock_perform_dict.items():
        df = df[(df['date'] >= start_calculate_date) & (df['date'] <= end_calculate_date)].reset_index(drop=True)
        portfolio_return_df[ticker] = df['portion_return'].fillna(0)

    portfolio_return_df['sum_return'] = portfolio_return_df.iloc[:,1:].sum(axis=1)

    return portfolio_return_df['sum_return']

##### Tính toán hiệu suất

- Tính toán bảng nắm giữ cổ phiếu

In [37]:
portfolio_raw_df = date_series.copy()
portfolio_raw_df['quarter'] = portfolio_raw_df['date'].apply(lambda x: next((key for key, value in quarter_timestamp_map_dict.items() if value[0] <= x <= value[1]), None))
portfolio_raw_df['phase'] = market_perform_df['market_phase_final']
portfolio_raw_df['phase_s1'] = portfolio_raw_df['phase'].shift(-1).fillna(0).astype(int)
portfolio_raw_df['phase_s2'] = portfolio_raw_df['phase'].shift(-2).fillna(0).astype(int)

portfolio_raw_df['return_cumsum5'] = market_perform_df['dummy_return'][::-1].rolling(window=5, min_periods=1).sum()[::-1]
portfolio_raw_df['return_cumsum20'] = market_perform_df['dummy_return'][::-1].rolling(window=20, min_periods=1).sum()[::-1]

portfolio_raw_df[['trend_5p','trend_20p']] = market_phase_df[['trend_5p','trend_20p']]

portfolio_raw_df['raw_holding_stock'] = portfolio_raw_df.apply(lambda x: signal_quarter_stock_list_dict[x['quarter']] if 
                                                        ((x['phase'] == 1) & (x['phase_s1'] == 0) & (x['phase_s2'] == 0)) |
                                                        ((x['phase'] == 1) & (x['phase_s1'] == 1) & (x['phase_s2'] == 0)) |
                                                        ((x['phase'] == 1) & (x['phase_s1'] == 1) & (x['phase_s2'] == 1)) |
                                                        ((x['phase'] == 0) & (x['phase_s1'] == 1) & (x['phase_s2'] == 1))
                                                    else [], axis=1) #else này bao gồm 100, 000 và 001

portfolio_raw_df['filtered_holding_stock'] = portfolio_raw_df.apply(lambda _: [], axis=1)
portfolio_raw_df['buy_signal'] = portfolio_raw_df.apply(lambda _: [], axis=1)
portfolio_raw_df['sell_signal'] = portfolio_raw_df.apply(lambda _: [], axis=1)

stock_classification_df = pd.read_excel("../.xlsx/full_stock_classification.xlsx", sheet_name='classification')

portfolio_list = [
    ('6stocks', 'small'),
    ('10stocks', 'small'),
    ('6stocks', 'medium'),
    ('6stocks', 'large')
]

portfolio_df_dict = {}
portfolio_holding_dict = {}
portfolio_perform_dict = {}
for porfolio_template, marketcap_template in portfolio_list:
    temp_portfolio_df, temp_holding_stock_dict = auto_portfolio(portfolio_raw_df, stock_dict, stock_classification_df, porfolio_template, marketcap_template)
    
    portfolio_name = f'{marketcap_template}_{porfolio_template}'
    portfolio_df_dict[portfolio_name] = temp_portfolio_df
    portfolio_holding_dict[portfolio_name] = temp_holding_stock_dict
    portfolio_perform_dict[portfolio_name] = calculate_stock_performance(temp_portfolio_df, temp_holding_stock_dict, stock_dict)

- Tạo lịch sử mua bán và thống kê lãi lỗ trung bình từng cổ phiếu

In [39]:
portfolio_trade_history_dict = {}
portfolio_mean_return_dict = {}
for portfolio_name, dict_values in portfolio_perform_dict.items():
    portfolio_trade_history_dict[portfolio_name] = extract_trade_history(dict_values, portfolio_name)
    portfolio_mean_return_dict[portfolio_name] = calculate_stock_mean_return(portfolio_trade_history_dict[portfolio_name], portfolio_name)

# Tổng hợp vào một bảng để tra soát
total_trade_history_df = pd.DataFrame()
for portfolio_name, dict_values in portfolio_trade_history_dict.items():
    total_trade_history_df = pd.concat([total_trade_history_df, dict_values], axis=0)

total_mean_return_df = pd.DataFrame()
for portfolio_name, dict_values in portfolio_mean_return_dict.items():
    total_mean_return_df = pd.concat([total_mean_return_df, dict_values], axis=0)

- Tạo lịch sử giao dịch cho từng ngày để xem mỗi ngày mua gì bán gì kèm tỉ trọng và mức giá

In [40]:
def expand_signals(df):
    rows = []
    for index, row in df.iterrows():
        date = row['date']
        if isinstance(row['buy_signal'], list):
            buy_signals = row['buy_signal']
        else:
            buy_signals_str = row['buy_signal'].strip('[]')
            buy_signals = [ticker.strip() for ticker in buy_signals_str.split(',')] if buy_signals_str else []
        if isinstance(row['sell_signal'], list):
            sell_signals = row['sell_signal']
        else:
            sell_signals_str = row['sell_signal'].strip('[]')
            sell_signals = [ticker.strip() for ticker in sell_signals_str.split(',')] if sell_signals_str else []
        for ticker in buy_signals:
            if ticker:  # Bỏ qua các ticker rỗng
                rows.append({'date': date,'ticker': ticker,'type': 'buy'})
        for ticker in sell_signals:
            if ticker:  # Bỏ qua các ticker rỗng
                rows.append({'date': date,'ticker': ticker,'type': 'sell'})
    # Tạo DataFrame mới
    return pd.DataFrame(rows)

In [41]:
portfolio_daily_trade_dict = {}
for portfolio_name, dict_values in portfolio_df_dict.items():
    value_df = dict_values[['date','buy_signal','sell_signal']]
    value_df['buy_signal'] = value_df['buy_signal'].shift(-1)
    value_df['sell_signal'] = value_df['sell_signal'].shift(-1)
    value_df.at[len(value_df)-1, 'buy_signal'] = []
    value_df.at[len(value_df)-1, 'sell_signal'] = []
    expanded_df = expand_signals(value_df)

    temp_df = pd.DataFrame()
    for i in range(len(expanded_df)):
        date_key = expanded_df.at[i, 'date']
        ticker = expanded_df.at[i, 'ticker']
        temp_stock_df = stock_dict[ticker][['date','open','high','low','close','trade_high', 'portion_high', 'trade_low', 'portion_low', 'trade_mean', 'portion_mean', 'trade_price']]
        temp_portion_df = portfolio_perform_dict[portfolio_name][ticker][['date', 'portion']]
        merge_row = expanded_df.iloc[[i]].merge(temp_stock_df[temp_stock_df['date'] == date_key], on='date').merge(temp_portion_df[temp_portion_df['date'] == date_key], on='date')

        temp_df = pd.concat([temp_df, merge_row], axis=0)

    portfolio_daily_trade_dict[portfolio_name] = temp_df[~((temp_df['type'] == 'buy') & (temp_df['portion'] == 0))].sort_values('date', ascending=False).reset_index(drop=True)

total_daily_trade_df = pd.DataFrame()
for portfolio_name, dict_values in portfolio_daily_trade_dict.items():
    temp_df = dict_values.copy()
    temp_df['name'] = portfolio_name
    total_daily_trade_df = pd.concat([total_daily_trade_df, temp_df], axis=0)

- Bảng quan sát cổ phiếu nắm giữ, cổ phiếu bán ra và cổ phiếu mua vào từng phiên

In [42]:
total_stock_list_df = pd.DataFrame()
for portfolio_name, dict_values in portfolio_df_dict.items():
    temp_df = dict_values[['date', 'phase', 'filtered_holding_stock', 'buy_signal', 'sell_signal', 'number_of_stock']]
    temp_df['filtered_holding_stock'] = temp_df['filtered_holding_stock'].astype(str)
    temp_df['buy_signal'] = temp_df['buy_signal'].astype(str)
    temp_df['sell_signal'] = temp_df['sell_signal'].astype(str)
    temp_df['name'] = portfolio_name
    total_stock_list_df = pd.concat([total_stock_list_df, temp_df], axis=0)


- Tính toán các bảng hiệu suất các danh mục

In [43]:
# Chọn khoảng thời gian tính toán
# start_calculate_date = '2022-11-01'
start_calculate_date = date_series.at[len(date_series)-1, 'date'].strftime('%Y-%m-%d')
end_calculate_date = date_series.at[0, 'date'].strftime('%Y-%m-%d')

In [44]:
current_year = datetime.now().year  # Will be 2025
year_map = {}

# Create yearly ranges from 2020 to current_year+1
for year in range(2020, current_year+1):  # Will go from 2020 to 2026
    start_date = pd.Timestamp(datetime(year, 1, 1)).strftime('%Y-%m-%d')
    end_date = pd.Timestamp(datetime(year, 12, 31)).strftime('%Y-%m-%d')
    year_map[year] = (start_date, end_date)

# Add 'total' entry spanning all years
total_start_date = pd.Timestamp(datetime(2020, 1, 1)).strftime('%Y-%m-%d')
total_end_date = pd.Timestamp(datetime(current_year, 12, 31)).strftime('%Y-%m-%d')
year_map['Total'] = (total_start_date, total_end_date)

In [45]:
total_perform_df = pd.DataFrame()
for year, time_range in year_map.items():

    start_calculate_date = time_range[0]
    end_calculate_date = time_range[1]
    temp_df = market_perform_df[(market_perform_df['date'] >= start_calculate_date) & (market_perform_df['date'] <= end_calculate_date)].reset_index(drop=True)

    temp_df['dummy_cum_return'] = temp_df['dummy_return'][::-1].cumsum()[::-1]
    temp_df['dummy_capital'] = calculate_capital(temp_df, 'dummy_return', 1000)

    temp_df['vnindex_cum_return'] = temp_df['vnindex_return'][::-1].cumsum()[::-1]
    temp_df['vnindex_capital'] = calculate_capital(temp_df, 'vnindex_return', 1000)


    for porfolio_template, marketcap_template in [('6stocks', 'small'), ('10stocks', 'small'), ('6stocks', 'medium'), ('6stocks', 'large')]:
        portfolio_name = f'{marketcap_template}_{porfolio_template}'
        try:
            temp_df[f'{portfolio_name}_return'] = calculate_portfolio_return(temp_df[['date']], portfolio_perform_dict[portfolio_name], start_calculate_date, end_calculate_date)
            temp_df[f'{portfolio_name}_cum_return'] = temp_df[f'{portfolio_name}_return'][::-1].cumsum()[::-1]
            temp_df[f'{portfolio_name}_capital'] = calculate_capital(temp_df, f'{portfolio_name}_return', 1000)
        except: #Thêm giữ liệu giả để có thể trình chiếu excel test
            temp_df[f'{portfolio_name}_return'] = 0
            temp_df[f'{portfolio_name}_cum_return'] = 0
            temp_df[f'{portfolio_name}_capital'] = 0

    temp_df['year'] = year
    total_perform_df = pd.concat([total_perform_df, temp_df], axis=0)


In [46]:
# print(len(small_6stocks_trade_history_df[small_6stocks_trade_history_df['return']<0]))
# print(len(small_6stocks_trade_history_df[small_6stocks_trade_history_df['return']>=0]))
# print(small_6stocks_trade_history_df[small_6stocks_trade_history_df['return']>=0]['return'].mean())
# print(small_6stocks_trade_history_df[small_6stocks_trade_history_df['return']<0]['return'].mean())

#### Lưu dữ liệu

In [47]:
# stock_perform_history = pd.DataFrame()
# for ticker, df in stock_dict.items():
#     temp_df = df.copy()
#     temp_df['market_phase'] = market_phase_df['market_phase_final']
#     temp_df['stock_return'] = calculate_pct_return(temp_df, 'market_phase')
#     temp_df['return_ratio'] = temp_df['stock_return'][::-1].cumsum()[::-1]

#     stock_perform_history = pd.concat([stock_perform_history, temp_df[['date', 'ticker', 'return_ratio']].iloc[[0]]], axis=0)
    
# stock_perform_history = stock_perform_history.merge(stock_classification_df[['ticker', 'exchange', 'fullname_industry', 'industry_perform']], on='ticker', how='left').sort_values('return_ratio', ascending=False)
# stock_perform_history.to_excel("../.xlsx/stock_perform_history.xlsx", index=False)

In [48]:
with pd.ExcelWriter("../visualize/test_assistant.xlsx", engine='openpyxl') as writer:
    total_perform_df.to_excel(writer, sheet_name='total_perform_df', index=False)
    total_trade_history_df.to_excel(writer, sheet_name='total_trade_history_df', index=False)
    total_mean_return_df.to_excel(writer, sheet_name='total_mean_return_df', index=False)
    total_daily_trade_df.to_excel(writer, sheet_name='total_daily_trade_df', index=False)
    total_stock_list_df.to_excel(writer, sheet_name='total_stock_list_df', index=False)